# PCMCI Analysis of Seascape dependence

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import rpy2
from scipy.stats import shapiro
import sklearn
import tigramite
import math
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

In [ ]:
file_dir=r'C:/Users/Mark/PycharmProjects/GitHub/Sea-scape-analysis/Data'
file_dir2=r'C:/Users/Mark/PycharmProjects/GitHub/Sea-scape-analysis'

In [ ]:
import seaborn as sns
from scipy.stats.mstats import winsorize

species_info=pd.read_csv(file_dir+"/spec_info.txt",sep="\t")
spec_list=["Eutrigla gurnardus","Amblyraja radiata","Limanda limanda", "Merlangius merlangus",
           "Clupea harengus","Gadus morhua","Sprattus sprattus","Callionymus lyra","Pleuronectes platessa"]

for species in spec_list:
    
    spec=species.replace(" ","_")
    
    print("## Lag 2 Winsorized 95% shuffle 3000")
    print(spec)

    my_data=pd.read_csv(file_dir+'/PCMCI_prep/%s_PCMCI_prep.csv'%spec,header=0,usecols=range(1,11))

    # winsorize data to reduce effect of extreme outliers on parcorr
    for column in my_data.columns:
        x=my_data[column]
        y=winsorize(pd.Series((x),dtype="float"),limits=[0.05,0.05])
        my_data[column]=y


    #transform to numpy nd-array
    my_data=my_data.to_numpy()
    np.min(my_data)
    np.max(my_data)
  
    #make sure no nan's
    my_data=np.nan_to_num(my_data)
    print(np.isnan(my_data).any())

    # Initialize PCMCI dataframe object from numpy array, specify time axis and variable names
    var_names = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10"]
    dataframe = pp.DataFrame(my_data, var_names=var_names) 
                             #datatime = np.arange(len(my_data)), 
                             #var_names=var_names)


    #define function
    parcorr = ParCorr(significance='shuffle_test',confidence="bootstrap",sig_samples=3000)
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=1)

    pcmci.verbosity = 1
    results = pcmci.run_pcmci(tau_max=2, pc_alpha=None)

    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')

    pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        conf_matrix=results["conf_matrix"],
        alpha_level = 0.05)